# Icons

Icons can be applied to both the `Title` of a `Panel` [widgets](./widgets.ipynb) and [commands](./commands.ipynb), providing more customization than `icon_class`.

In [ ]:
from ipylab import Icon

## SVG

An icon requires both a _unique_ name, as well as an SVG string. There are some [guidelines](https://jupyterlab.readthedocs.io/en/stable/extension/ui_components.html#labicon-set-up-and-render-icons) for creating "good" icons. For example:
- don't include the `<?xml>` declaration
- don't use `ids`
- don't specify a `width` or `height`
  - ensures the icon can be used in a number of settings
- use the `jp-icon*` classes on filled items
  - ensures the icon looks good on light and dark themes

In [ ]:
SVG = """<svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24">
  <circle class="jp-icon-selectable jp-icon3" cx="12" cy="12" r="12" fill="#616161" />
  <path class="jp-contrast0" fill="#fff" d="M9 16.17L4.83 12l-1.42 1.41L9 19 21 7l-1.41-1.41z"/>
</svg>"""

Icons can be displayed directly, and sized with the `layout` member inherited from `ipywidgets.DOMWidget`.

In [ ]:
icon = Icon(name="my-icon", svgstr=SVG, layout=dict(width="32px"))
icon

### More about `jp-icon` classes
The interactive below isn't particuarly _robust_, but shows how the different `jp-icon-*` classes can be used.

In [ ]:
from ipylab import Panel, JupyterFrontEnd
from ipywidgets import SelectionSlider, FloatSlider, VBox
from traitlets import dlink, link

icon_prefix = ["", "-accent", "-brand", "-contrast", "-warn"]
options = [""] + [f"jp-icon{sub}{i}" for sub in icon_prefix for i in range(5)]
background = SelectionSlider(description="background", options=options)
foreground = SelectionSlider(description="foreground", options=options)

repaint = lambda: SVG.replace("jp-icon3", background.value).replace("jp-contrast0", foreground.value)

dlink((background, "value"), (icon, "svgstr"), lambda x: SVG.replace("jp-icon3", x))
dlink((foreground, "value"), (icon, "svgstr"), lambda x: SVG.replace("jp-contrast0", x))
size = FloatSlider(32, description="size")
dlink((size, "value"), (icon.layout, "width"), "{}px".format)
icon_controls = VBox([background, foreground, size, icon])
icon_controls

## Icons on Panel Titles

Once defined, an icon can be used on a panel title in place of `icon_class` 

In [ ]:
app = JupyterFrontEnd()
panel = Panel([icon_controls])
panel.title.icon = icon
dlink((background, "value"), (panel.title, "label"))
app.shell.add(panel, "main", {"mode": "split-right"})

## Icons on Commands

Icons can also assigned to [commands](./commands.ipynb) to provide additional context. 

In [ ]:
import asyncio
import random

async def randomize_icon():
    for i in range(10):
        background.value = random.choice(options)
        await asyncio.sleep(0.1)

In [ ]:
app.commands.add_command(
    "my-icon:randomize",
    lambda: asyncio.get_running_loop().create_task(randomize_icon()),
    label="Randomize My Icon",
    icon=icon
)

To see these, add the a _Command Palette_ with the same `command_id`:

In [ ]:
from ipylab.commands import CommandPalette

palette = CommandPalette()
palette.add_item("my-icon:randomize", "All My Commands")

Then open the _Command Palette_.

In [ ]:
app.commands.execute("apputils:activate-command-palette")